<a href="https://colab.research.google.com/github/Ben-Gayther/FineTunedQ-A/blob/main/InitialTestingColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/si

In [ ]:
# Import required libraries
import os
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
import datasets
import torch
import torch.nn as nn

In [ ]:
dataset_name = "flax-sentence-embeddings/stackexchange_titlebody_best_and_down_voted_answer_jsonl"
ai_dataset = datasets.load_dataset(dataset_name, 'ai')['train']
ds_dataset = datasets.load_dataset(dataset_name, 'datascience')['train']
se_dataset = datasets.load_dataset(dataset_name, 'softwareengineering')['train']

Generating train split: 0 examples [00:00, ? examples/s]

Dataset stackexchange_titlebody_best_and_down_voted_answer_jsonl downloaded and prepared to /root/.cache/huggingface/datasets/flax-sentence-embeddings___stackexchange_titlebody_best_and_down_voted_answer_jsonl/ai/1.1.0/a767719a162391b61f7fecca12b41572102b8cf2909d9c06f55eb7a70c7aa579. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset stackexchange_titlebody_best_and_down_voted_answer_jsonl downloaded and prepared to /root/.cache/huggingface/datasets/flax-sentence-embeddings___stackexchange_titlebody_best_and_down_voted_answer_jsonl/datascience/1.1.0/a767719a162391b61f7fecca12b41572102b8cf2909d9c06f55eb7a70c7aa579. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset stackexchange_titlebody_best_and_down_voted_answer_jsonl downloaded and prepared to /root/.cache/huggingface/datasets/flax-sentence-embeddings___stackexchange_titlebody_best_and_down_voted_answer_jsonl/softwareengineering/1.1.0/a767719a162391b61f7fecca12b41572102b8cf2909d9c06f55eb7a70c7aa579. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Combine the splits into a single dataset
combined_dataset = datasets.concatenate_datasets([ai_dataset, ds_dataset, se_dataset])

# Print the number of examples in the combined dataset
print(len(combined_dataset))

4693


In [ ]:
del ai_dataset, ds_dataset, se_dataset


In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [ ]:
# Define preprocessing function
# https://tsmatz.wordpress.com/2022/12/12/huggingface-japanese-question-answering/
# https://keras.io/examples/nlp/question_answering/
def preprocess_function(examples):
    # Tokenize the contexts and questions (as pairs of inputs)
    inputs = tokenizer(examples["title_body"], examples["upvoted_answer"], truncation=True, max_length=512, padding="max_length", return_offsets_mapping=True)
    print(inputs.keys())
    return inputs

In [ ]:
preprocessed_datasets = combined_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4693 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])
dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])
dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])
dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])
dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])


In [ ]:
# Split the dataset into train and validation
seed = 42

# 80% of the dataset will be used for training
total_num_examples = len(preprocessed_datasets)
train_num_examples = int(0.8 * total_num_examples)

# Shuffle the dataset
preprocessed_datasets = preprocessed_datasets.shuffle(seed=seed)

# Split the dataset
train_dataset = preprocessed_datasets.select(range(train_num_examples))
validation_dataset = preprocessed_datasets.select(range(train_num_examples, total_num_examples))


In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    "test-qa",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
class QuestionAnsweringTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        start_logits, end_logits = outputs.start_logits, outputs.end_logits
        start_positions, end_positions = inputs["start_positions"], inputs["end_positions"]
        loss_fct = nn.CrossEntropyLoss()
        start_loss = loss_fct(start_logits, start_positions)
        end_loss = loss_fct(end_logits, end_positions)
        total_loss = (start_loss + end_loss) / 2
        return (total_loss, outputs) if return_outputs else total_loss

In [ ]:
# Define the trainer
trainer = QuestionAnsweringTrainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("test-qa")

The following columns in the training set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: upvoted_answer, offset_mapping, title_body, downvoted_answer. If upvoted_answer, offset_mapping, title_body, downvoted_answer are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3754
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 705
  Number of trainable parameters = 66364418


KeyError: ignored